In [27]:
import requests
import re
import os
import pandas as pd
import shutil
import logging
import psycopg2

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def fetch_data(url):
    data = []
    
    if os.path.exists("Image") and os.path.isdir("Image"):
        shutil.rmtree("Image")
        logging.info("Папка 'Image' со старыми данными была удалена")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        response_text = response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка при выполнении запроса для {url}: {e}")
        return pd.DataFrame(columns=["name", "description", "image", "town", "type"])
    
    pattern = re.compile(
        r'{"name":"(?P<name>.*?)",.*?"description":"(?P<description>.*?)",.*?"image":"(?P<image>.*?)"}'
    )
    
    try:
        for match in pattern.finditer(response_text):
            name = match.group("name")
            description = match.group("description")
            image = match.group("image")
            if name and description and image:
                data.append({
                    "name": name,
                    "description": description,
                    "image": image
                })
            else:
                logging.warning(f"Пропущена запись из-за отсутствующих данных для {url}: {match.groupdict()}")
    except Exception as e:
        logging.error(f"Ошибка при парсинге данных для {url}: {e}")
        return pd.DataFrame(columns=["name", "description", "image", "town", "type"])

    try:
        df = pd.DataFrame(data, columns=["name", "description", "image"])
        if df.empty:
            logging.warning(f"Данные не найдены или не удалось их распарсить для {url}.")
        else:
            if "spb" in url:
                df["town"] = "Санкт-Петербург"
            else:
                df["town"] = "Москва"
            if "cinema" in url:
                df["type"] = "cinema"
            elif "theatre" in url:
                df["type"] = "theatre"
            elif "concerts" in url:
                df["type"] = "concerts"
            elif "exhibitions" in url:
                df["type"] = "exhibitions"
        return df
    except Exception as e:
        logging.error(f"Ошибка при создании DataFrame для {url}: {e}")
        return pd.DataFrame(columns=["name", "description", "image", "town", "type"])

urls = [
    "https://www.afisha.ru/spb/cinema/",
    "https://www.afisha.ru/spb/theatre/",
    "https://www.afisha.ru/spb/concerts/",
    "https://www.afisha.ru/spb/exhibitions/",
    "https://www.afisha.ru/msk/cinema/",
    "https://www.afisha.ru/msk/theatre/",
    "https://www.afisha.ru/msk/concerts/"
]

parsed_data = pd.DataFrame()
for url in urls:
    df = fetch_data(url)
    parsed_data = pd.concat([parsed_data, df], ignore_index=True)

logging.info("Завершена обработка всех URL")

2025-01-05 20:38:15,698 - INFO - Папка 'Image' со старыми данными была удалена


2025-01-05 20:38:35,899 - INFO - Завершена обработка всех URL


In [28]:
def make_correct_url(url):
    match = re.search(r'(https?://\S+\.(?:jpg|jpeg|png|gif|webp))', url)
    if match:
        return match.group(1)
    return None

def clean_df(df):
    def has_unwanted_chars(description):
        if not isinstance(description, str):
            return False
        if re.search(r"<[^>]*>", description):
            return True
        return False
    df_cleaned = df[~df['description'].apply(has_unwanted_chars)]
    return df_cleaned

clean_data = (clean_df(parsed_data)
              .assign(image=lambda x: x['image'].apply(make_correct_url))
              .drop_duplicates(subset=['name', 'town', 'type'])
              .reset_index(drop=True))

In [29]:
image_folder = "Image"
paths = []

if not os.path.exists(image_folder):
    os.makedirs(image_folder)

def download_image(url, folder, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()

        file_path = os.path.join(folder, filename)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        return file_path
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка при скачивании {url}: {e}")
        if response is not None:
            logging.error(f"Ответ сервера: {response.status_code} - {response.text[:100]}")
        return None
    
for index, row in clean_data.iterrows():
    image_url = row["image"]
    if pd.notna(image_url) and isinstance(image_url, str):
        if image_url:
            filename = f"image_{index}.jpg"
            file_path = download_image(image_url, image_folder, filename)
            if file_path:
                paths.append(file_path)
            else:
                paths.append(None)
        else:
            logging.warning(f"Некорректный URL: {image_url}")
            paths.append(None)
    else:
        paths.append(None)

clean_data['path'] = paths
clean_data.drop(columns=['image'], inplace=True)

logging.info("Обработка завершена")

2025-01-05 20:40:11,715 - INFO - Обработка завершена


In [30]:
connection_line = "dbname='events' user='postgres' password='postgres' host='localhost' sslmode='disable'"

try:
    conn = psycopg2.connect(connection_line)
    cursor = conn.cursor()
    logging.info("Подключение к базе данных успешно выполнено")
except Exception as e:
    logging.error(f"Ошибка подключения к базе данных: {e}")

query = """
CREATE TABLE IF NOT EXISTS events (
    name VARCHAR(500) NOT NULL,
    description VARCHAR(500) NOT NULL,
    town VARCHAR(500) NOT NULL,
    type VARCHAR(500) NOT NULL,
    path VARCHAR(500) NOT NULL
);
TRUNCATE TABLE events;
"""

try:
    cursor.execute(query)
    conn.commit()
    logging.info("Таблица создана или уже существует")
except Exception as e:
    logging.error(f"Ошибка при создании таблицы: {e}")

def insert_data(df, conn):
    try:
        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO events (name, description, town, type, path)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['name'], row['description'], row['town'], row['type'], row['path']))
        conn.commit()
        logging.info("Данные успешно записаны в базу данных")
    except Exception as e:
        logging.error(f"Ошибка при записи данных: {e}")
        conn.rollback()

insert_data(clean_data, conn)

cursor.close()
conn.close()
logging.info("Соединение с базой данных закрыто")

2025-01-05 20:41:47,441 - INFO - Подключение к базе данных успешно выполнено
2025-01-05 20:41:47,456 - INFO - Таблица создана или уже существует
2025-01-05 20:41:47,559 - INFO - Данные успешно записаны в базу данных
2025-01-05 20:41:47,560 - INFO - Соединение с базой данных закрыто
